In [87]:
import math
import string
import random
import time
from typing import List


def tokenize(text: str) -> List[str]:
    """
    :param text: Takes input sentence
    :return: tokenized sentence
    """
    for punct in string.punctuation:
        text = text.replace(punct, " " + punct + " ")
    t = text.split()
    return t


class NGramModelWithBackoff:
    def n_gram_counts(self, text, n):
        n_tokens = len(text)
        result = dict()
        for i in range(n_tokens - n + 1):
            key = tuple(text[i : i + n])
            if key in result:
                result[key] += 1
            else:
                result[key] = 1
        return result

    def __init__(self, text):
        self.uni_counts = self.n_gram_counts(text, 1)
        self.bi_counts = self.n_gram_counts(text, 2)
        self.tri_counts = self.n_gram_counts(text, 3)
        self.four_counts = self.n_gram_counts(text, 4)
        self.total_count = sum(self.uni_counts.values())
        self.vocab = len(self.uni_counts)

    def _get_probability(self, ngram):
        if len(ngram) == 1:
            return self.uni_counts.get(ngram, 0) / self.total_count
        elif len(ngram) == 2:
            return self.bi_counts.get(ngram, 0) / self.uni_counts.get(ngram[0], 1)
        elif len(ngram) == 3:
            return self.tri_counts.get(ngram, 0) / self.bi_counts.get(ngram[:2], 1)
        elif len(ngram) == 4:
            return self.four_counts.get(ngram, 0) / self.tri_counts.get(ngram[:3], 1)
        else:
            return 0

    def generate_starting_trigram(self):
        # Start with the most frequent word based on unigram counts
        first_word = random.choices(
            list(self.uni_counts.keys()), weights=self.uni_counts.values()
        )[0]

        # Check if there are bigrams matching the first word
        matching_bigrams = [
            word[1] for word in self.bi_counts.keys() if word[0] == first_word
        ]

        # If there are matching bigrams, select the most common second word
        if matching_bigrams:
            second_word = max(
                matching_bigrams, key=lambda x: self.bi_counts.get((first_word, x), 0)
            )
        else:
            # If no matching bigrams, select a random second word from the corpus
            second_word = random.choice(list(self.uni_counts.keys()))

        # Use the bigram along with the preceding word from trigram counts to form a trigram
        matching_trigrams = [
            word[2]
            for word in self.tri_counts.keys()
            if word[:2] == (first_word, second_word)
        ]
        if matching_trigrams:
            third_word = max(
                matching_trigrams,
                key=lambda x: self.tri_counts.get((first_word, second_word, x), 0),
            )
        else:
            # If no matching trigrams, select a random third word from the corpus
            third_word = random.choice(list(self.uni_counts.keys()))

        return (first_word, second_word, third_word)

    def generate_text(self, length):
        text = tuple()
        starting_trigram = self.generate_starting_trigram()

        for _ in range(length - 2):
            next_word = self.generate_next_word(starting_trigram)
            text += starting_trigram
            text += (next_word,)
            starting_trigram = text[-2:]

        return " ".join([" ".join(token) for token in text])

    # def generate_next_word(self, given_trigram):
    #     max_count = 0
    #     next_word = None

    #     # Check if the given trigram exists in the four-grams
    #     for four_gram, count in self.four_counts.items():
    #         if four_gram[:3] == given_trigram:
    #             # Update the most common next word if count is greater
    #             if count > max_count:
    #                 max_count = count
    #                 next_word = four_gram[3]

    #     # Check if the given trigram exists in the trigrams
    #     for tri_gram, count in self.tri_counts.items():
    #         if tri_gram[:2] == given_trigram[1:]:
    #             # Update the most common next word if count is greater
    #             if count > max_count:
    #                 max_count = count
    #                 next_word = tri_gram[2]

    #     # Check if the given trigram exists in the bigrams
    #     for bi_gram, count in self.bi_counts.items():
    #         if bi_gram[:1] == given_trigram[2:]:
    #             # Update the most common next word if count is greater
    #             if count > max_count:
    #                 max_count = count
    #                 next_word = bi_gram[1]

    #     # If none of the above cases matched, return the most common word based on unigram counts
    #     if next_word is None:
    #         next_word = max(self.uni_counts, key=self.uni_counts.get)

    #     return next_word
    def generate_next_word(self, given_trigram):
        # Collect all possible next words and their probabilities
        possible_next_words = []
        total_weight = 0
        for word in self.uni_counts.keys():
            prob = self._get_probability(given_trigram + (word,))
            total_weight += prob
            possible_next_words.append((word, prob))

        # Add a small epsilon to the probabilities to ensure they are all greater than zero
        epsilon = 1e-6
        for i in range(len(possible_next_words)):
            possible_next_words[i] = (
                possible_next_words[i][0],
                possible_next_words[i][1] + epsilon,
            )

        # Choose the next word based on probabilities
        next_word = random.choices(
            [word for word, _ in possible_next_words],
            weights=[prob for _, prob in possible_next_words],
        )[0]

        return next_word

    def calculate_perplexity(self, text):
        log_prob_sum = 0
        N = len(text)
        for i in range(2, N):
            context = tuple(text[i - 2 : i])
            next_word = text[i]
            prob = self._get_probability(context + (next_word,))
            # Avoid taking log of zero by adding a small epsilon
            log_prob_sum += math.log(prob + 1e-10)

        # Compute perplexity using the formula
        perplexity = math.exp(-log_prob_sum / (N - 2))
        return perplexity


with open("Moby_dick.txt", "r", encoding="utf-8") as f:
    text = f.read()
    text = tokenize(text)

m = NGramModelWithBackoff(text)
print(m.generate_text(20))
# print(m.calculate_perplexity())

upheaving ultimate continental knocks continental knocks rooms knocks rooms Cathedral rooms Cathedral hearts Cathedral hearts 30 hearts 30 paintings 30 paintings northward paintings northward story northward story parlor story parlor oftentimes parlor oftentimes Book oftentimes Book Vedas Book Vedas referring Vedas referring trophies referring trophies considerably trophies considerably thunder considerably thunder walrus thunder walrus slipping


In [88]:
import random

# Open and read the text file
with open("Moby_dick.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Tokenize the text
text_tokens = tokenize(text)

# Shuffle the tokens to randomize the order
random.shuffle(text_tokens)

# Define the split ratio (e.g., 80% for training, 20% for testing)
split_ratio = 0.8
split_index = int(len(text_tokens) * split_ratio)

# Split the tokens into training and test sets
train_tokens = text_tokens[:split_index]
test_tokens = text_tokens[split_index:]

# Create an instance of the NGramModelWithProbabilities class
m_with_prob = NGramModelWithBackoff(train_tokens)

# Calculate perplexity using the test set
perplexity = m_with_prob.calculate_perplexity(test_tokens)
print("Perplexity on test set:", perplexity)

Perplexity on test set: 1902654688.830192
